In [55]:
import ultralytics
ultralytics.__version__

'8.0.106'

In [56]:
import torch
torch.__version__

'1.7.1+cu110'

In [57]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650'

# Detect, track and count Persons

In [58]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np



In [59]:
from keras.models import load_model
age_model = load_model('adchcheck.h5')

def get_age(frame):
    try:
        frame = cv2.resize(frame, (150, 150))  # Resize to match model input size
        frame = frame.astype('float32') / 255.0  # Normalize if required
        frame = np.expand_dims(frame, axis=0)
        prediction = age_model.predict(frame)
        if prediction[0] > 0.9:
            return "Child"
        else:
            return "Therapist"

    except cv2.error as e:
        print(f"OpenCV Error: {e}")
        # Handle the error or continue the execution
        pass
    
    except Exception as e:
        print(f"Unexpected Error: {e}")
        # Handle the error or continue the execution
        pass
    
    # Continue with the rest of your code
    print("Continuing execution...")

# DeepSORT

In [60]:
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [61]:
# Define the video path
video_path = "test vids/2.mp4"

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 30

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output/out11.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [62]:
frames = []

unique_track_ids = set()

In [63]:
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        

        model = YOLO("weights/yolov8n.pt", "v8")  # load a pretrained model (recommended for training)

        results = model(frame, device=0, classes=0, conf=0.8)

        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                #print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, frame)
        

        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height

            # Set color values for red, blue, and green
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color
            
            if class_name == 'person':

                xmin = int(x1)
                xmax = int(x1 + w)
                ymin = int(y1)
                ymax = int(y1 + h)

                person_crop = frame[ymin:ymax, xmin:xmax]
                age = get_age(person_crop)

                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
                

                text_color = (0, 0, 0)  # Black color for text
                cv2.putText(frame, f"{class_name}-{track_id}--{age}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

                # Add the track_id to the set of unique track IDs
                unique_track_ids.add(track_id)
            
            else:
                pass

        # Update the person count based on the number of unique track IDs
        person_count = len(unique_track_ids)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time

        # Draw person count on frame
        # cv2.putText(frame, f"Person Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Append the frame to the list
        frames.append(frame)

        # Write the frame to the output video file
        out.write(frame)

        # Show the frame
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 2 persons, 18.3ms
Speed: 4.1ms preprocess, 18.3ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 persons, 25.6ms
Speed: 4.0ms preprocess, 25.6ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 persons, 17.3ms
Speed: 3.0ms preprocess, 17.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 persons, 17.0ms
Speed: 3.4ms preprocess, 17.0ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 persons, 17.2ms
Speed: 2.0ms preprocess, 17.2ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 persons, 17.8ms
Speed: 2.7ms preprocess, 17.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 persons, 16.6ms
Speed: 3.2ms preprocess, 16.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 persons, 21.1ms
Speed: 3.5ms preprocess, 21.1ms inference, 2.3ms postprocess per image at shape (

OpenCV Error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Continuing execution...



0: 384x640 3 persons, 15.2ms
Speed: 3.0ms preprocess, 15.2ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)


OpenCV Error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Continuing execution...



0: 384x640 3 persons, 16.1ms
Speed: 2.0ms preprocess, 16.1ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)


...

0: 384x640 (no detections), 19.3ms
Speed: 2.0ms preprocess, 19.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



OpenCV Error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Continuing execution...


0: 384x640 (no detections), 20.4ms
Speed: 3.0ms preprocess, 20.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



OpenCV Error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Continuing execution...


0: 384x640 (no detections), 26.2ms
Speed: 2.5ms preprocess, 26.2ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 21.2ms
Speed: 3.5ms preprocess, 21.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 18.2ms
Speed: 2.3ms preprocess, 18.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 21.1ms
Speed: 3.2ms preprocess, 21.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 17.1ms
Speed: 2.5ms preprocess, 17.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 17.3ms
Speed: 3.0ms preprocess, 17.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 20.0ms
Speed: 2.7ms preprocess, 20.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 23.1ms
Speed: 4.0ms preprocess, 23.1ms in